In [1]:
# install openpyxl in env
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
data = pd.read_excel("problem_data_new.xlsx")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33860 entries, 0 to 33859
Data columns (total 50 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ARCHIVE_DATE                   33860 non-null  object 
 1   SNAPSHOT_NUMBER                33860 non-null  int64  
 2   EBS_JOB_NUMBER                 33860 non-null  object 
 3   JOB_STATUS                     33860 non-null  object 
 4   SCHEDULED_DATE                 33860 non-null  object 
 5   JOB_CREATION_DATE              33860 non-null  object 
 6   FG_CODE                        33860 non-null  object 
 7   PLANT                          33860 non-null  object 
 8   LINE_CODE                      33860 non-null  object 
 9   SCHEDULED_PRODUCTION_QUANTITY  33860 non-null  float64
 10  QUANTITY_START                 33860 non-null  float64
 11  RM_ITEM_CODE                   33860 non-null  object 
 12  RM_REQUIRED                    33860 non-null 

In [5]:
data.head(15)

,ARCHIVE_DATE,SNAPSHOT_NUMBER,EBS_JOB_NUMBER,JOB_STATUS,SCHEDULED_DATE,JOB_CREATION_DATE,FG_CODE,PLANT,LINE_CODE,SCHEDULED_PRODUCTION_QUANTITY,QUANTITY_START,RM_ITEM_CODE,RM_REQUIRED,RM_INVENTORY,ID,WEEK,JOB_CREATION_FLAG,INVENTORY_DAY_BUCKET,AVAILABLE_INVENTORY,RM Item Coverage,FC_explain,FC_explain_cf,FC_explain_nc,Scheduled-Start,Scheduled-Start_%,Job Coverage,Covered,Covered Flag,Not Covered,Cases per Pallet,EPQ,SS,Scheduled-EPQ,Scheduled-EPQ_%,Low,Medium,High,Spring,Spring Specialty,Spring Non-Specialty,Freeze,Plant,LMS,lms_yes,lms_no,lms_int,lms_fly,D_SCH_ST,SEPQ,TARGET
0,6/24/2020 0:00,1,MPS00552352,Closed,6/24/2020 16:50,2020-09-06 07:44:00,GRV05L24PDMN,ABQ,AB1,25788.0,26788.0,SHV09GRV0524DMN1A,25788.0,0,291149,26,OUTSIDE LOCK WIDNOW,6/24/2020,120306.0,Covered,2,0,0,-1000.0,-0.038778,Covered,1,0,0,84,8316.0,8820.0,17472.0,0.677524,1.0,0.0,0.0,0,0,0,0,ABQ,No,0,1,0,0,0.003257,0.677524,1
1,2020-04-03 00:00:00,1,L00513966,Closed,2020-04-03 12:25:00,2/23/2020 21:50,KRK05L40PDMGPN,ABQ,AB1,20304.0,30000.0,SHV11KRK0540DMNA,20304.0,0,290927,10,OUTSIDE LOCK WIDNOW,2020-04-03 00:00:00,96931.0,Covered,2,0,0,-9696.0,-0.477541,Covered,1,0,0,48,16416.0,5520.0,3888.0,0.191489,1.0,0.0,0.0,0,0,0,0,ABQ,No,0,1,0,0,0.196217,0.191489,0
2,5/20/2020 0:00,1,L00546028,Closed,5/20/2020 12:10,2020-11-05 11:43:00,NDW05L32PDRCHN60,ABQ,AB1,28200.0,31560.0,SHV23NDW0532DRN,22680.0,0,290661,21,OUTSIDE LOCK WIDNOW,5/20/2020,81315.0,Covered,2,0,0,-3360.0,-0.119149,Covered,1,0,0,60,6000.0,1260.0,22200.0,0.787234,1.0,0.0,0.0,0,0,0,1,ABQ,No,0,1,0,0,0.119149,0.787234,0
3,5/22/2020 0:00,1,MPS00536085,Closed,5/22/2020 12:10,2020-08-05 06:48:00,KRK05L40PDMGPN,ABQ,AB1,24432.0,24816.0,SHV11KRK0540DMNA,24432.0,0,290971,21,OUTSIDE LOCK WIDNOW,5/22/2020,126858.0,Covered,2,0,0,-384.0,-0.015717,Covered,1,0,0,48,16416.0,5520.0,8016.0,0.328094,1.0,0.0,0.0,0,0,0,0,ABQ,No,0,1,0,0,0.015717,0.328094,0
4,5/23/2020 0:00,1,MPS00536083,Closed,5/23/2020 4:45,2020-08-05 06:48:00,GRV05L40PDMN,ABQ,AB1,57840.0,58840.0,SHV03GRV0540DMNA,57840.0,0,291808,21,OUTSIDE LOCK WIDNOW,5/23/2020,0.0,Covered Flag,0,2,0,-1000.0,-0.017289,Covered Flag,0,1,0,48,45024.0,34080.0,12816.0,0.221577,1.0,0.0,0.0,0,0,0,0,ABQ,No,0,1,0,0,0.003320,0.221577,0
5,2020-09-02 00:00:00,1,L00504335,Closed,2020-09-02 17:50:00,1/26/2020 21:24,KRK05L40PDMGPN,ABQ,AB1,10032.0,12960.0,SHV11KRK0540DMNA,10032.0,0,290907,7,OUTSIDE LOCK WIDNOW,2020-09-02 00:00:00,145406.0,Covered,2,0,0,-2928.0,-0.291866,Covered,1,0,0,48,16416.0,5520.0,-6384.0,-0.636364,1.0,0.0,0.0,0,0,0,1,ABQ,No,0,1,0,0,0.196172,0.636364,0
6,7/31/2020 0:00,1,L00566486,Closed,7/31/2020 15:45,7/21/2020 9:18,GRV05L40PDMN,ABQ,AB1,116256.0,132432.0,SHV03GRV0540DMNA,116256.0,0,291854,31,OUTSIDE LOCK WIDNOW,7/31/2020,502089.0,Covered,2,0,0,-16176.0,-0.139141,Covered,1,0,0,48,45024.0,34080.0,71232.0,0.612717,1.0,0.0,0.0,0,0,0,1,ABQ,No,0,1,0,0,0.139141,0.612717,0
7,2020-09-04 00:00:00,1,L00529412,Closed,2020-09-04 05:55:00,3/30/2020 15:19,GRV05L40PDMN,ABQ,AB1,50016.0,67240.0,SHV03GRV0540DMNA,46656.0,0,291777,15,OUTSIDE LOCK WIDNOW,2020-09-04 00:00:00,375198.0,Covered,2,0,0,-17224.0,-0.344370,Covered,1,0,0,48,45024.0,34080.0,4992.0,0.099808,1.0,0.0,0.0,0,0,0,0,ABQ,No,0,1,0,0,0.333973,0.099808,0
8,7/22/2020 0:00,1,L00562455,Closed,7/22/2020 17:20,2020-07-07 15:56:00,MMK05L45PDMN,ABQ,AB1,27846.0,28056.0,SHV04MMK0545DMNA,27846.0,0,291581,30,OUTSIDE LOCK WIDNOW,7/22/2020,129700.0,Covered,2,0,0,-210.0,-0.007541,Covered,1,0,0,42,7098.0,11676.0,20748.0,0.745098,1.0,0.0,0.0,0,0,0,1,ABQ,No,0,1,0,0,0.007541,0.745098,0
9,6/29/2020 0:00,1,L00555708,Closed,6/29/2020 10:50,6/15/2020 8:31,NDW05L24PDRCHN84,ABQ,AB1,17892.0,42924.0,SHV14NDW0524DRN1A,40488.0,0,290804,27,OUTSIDE LOCK WIDNOW,6/29/2020,156169.0,Covered,2,0,0,-25032.0,-1.399061,Covered,1,0,0,84,41664.0,5964.0,-23772.0,-1.328638,1.0,0.0,0.0,0,0,0,1,ABQ,No,0,1,0,0,1.375587,1.328638,0


In [6]:
data.nunique()

ARCHIVE_DATE                       252
SNAPSHOT_NUMBER                      1
EBS_JOB_NUMBER                   33860
JOB_STATUS                           3
SCHEDULED_DATE                   25875
JOB_CREATION_DATE                 7937
FG_CODE                           1032
PLANT                               34
LINE_CODE                          105
SCHEDULED_PRODUCTION_QUANTITY     5897
QUANTITY_START                    7806
RM_ITEM_CODE                       951
RM_REQUIRED                       7077
RM_INVENTORY                         1
ID                               33860
WEEK                                40
JOB_CREATION_FLAG                    3
INVENTORY_DAY_BUCKET               265
AVAILABLE_INVENTORY              26957
RM Item Coverage                     3
FC_explain                           6
FC_explain_cf                        5
FC_explain_nc                        5
Scheduled-Start                   5054
Scheduled-Start_%                13969
Job Coverage             

In [7]:
# install jinja2 in environment before running this cell
data.isna().sum().to_frame('missing_count').assign(missing_pct = lambda x: x.missing_count / data.shape[0]).sort_values('missing_pct', ascending=False).query('missing_count>0').style.format('{:.2%}', subset=['missing_pct'])

,missing_count,missing_pct
EPQ,2554,7.54%
Scheduled-EPQ_%,2554,7.54%
SEPQ,2554,7.54%
Scheduled-EPQ,2554,7.54%
SS,2350,6.94%
Medium,1284,3.79%
High,1284,3.79%
Low,1284,3.79%


In [8]:
data = data.drop(columns=['EBS_JOB_NUMBER', 'ID', 'SNAPSHOT_NUMBER', 'RM_INVENTORY', 'PLANT'])

In [9]:
data[['ARCHIVE_DATE', 'SCHEDULED_DATE', 'JOB_CREATION_DATE', 'INVENTORY_DAY_BUCKET']] = data[['ARCHIVE_DATE', 'SCHEDULED_DATE', 'JOB_CREATION_DATE', 'INVENTORY_DAY_BUCKET']].apply(pd.to_datetime)

In [10]:
data_num_stats = data.describe(exclude=['object', 'datetime64'])
data_num_stats = pd.concat([data_num_stats, data.select_dtypes(exclude=['object', 'datetime64']).reindex(data_num_stats.columns, axis = 1).agg(['median', 'skew', 'kurtosis'])])
data_num_stats

,SCHEDULED_PRODUCTION_QUANTITY,QUANTITY_START,RM_REQUIRED,WEEK,AVAILABLE_INVENTORY,FC_explain,FC_explain_cf,FC_explain_nc,Scheduled-Start,Scheduled-Start_%,Covered,Covered Flag,Not Covered,Cases per Pallet,EPQ,SS,Scheduled-EPQ,Scheduled-EPQ_%,Low,Medium,High,Spring,Spring Specialty,Spring Non-Specialty,Freeze,lms_yes,lms_no,lms_int,lms_fly,D_SCH_ST,SEPQ,TARGET
count,3.386000e+04,3.386000e+04,33860.000000,33860.000000,3.386000e+04,33860.000000,33860.000000,33860.000000,33860.000000,33860.000000,33860.000000,33860.000000,33860.000000,33860.000000,31306.000000,31510.000000,31306.000000,31306.000000,32576.000000,32576.000000,32576.000000,33860.000000,33860.000000,33860.000000,33860.000000,33860.000000,33860.000000,33860.000000,33860.000000,33860.000000,31306.000000,33860.000000
mean,2.449780e+04,2.682506e+04,19454.171292,20.434613,1.546158e+05,1.710219,0.318045,0.036562,-2327.267516,-0.760189,0.795393,0.180626,0.023981,63.290992,15400.382163,7754.732402,5327.486742,-0.787896,0.570696,0.045555,0.383749,0.110455,0.008949,0.101506,0.426757,0.342410,0.544477,0.065239,0.047874,0.807869,1.508157,0.344448
std,4.326193e+04,4.556004e+04,25155.497595,10.807956,2.144682e+05,0.800386,0.720481,0.250269,15126.826354,18.891933,0.403420,0.384714,0.152992,29.145823,17360.304698,10787.572822,25732.290263,21.976232,0.494984,0.208521,0.486305,0.313460,0.094174,0.302002,0.494614,0.474523,0.498025,0.246951,0.213502,18.273069,21.938573,0.475195
min,-8.304000e+03,1.000000e+00,-4800.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000,-611328.000000,-2207.000000,0.000000,0.000000,0.000000,0.000000,22.000000,0.000000,-172600.000000,-3227.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-201.760000,-84.160000,0.000000
25%,7.056000e+03,8.446500e+03,5040.000000,12.000000,1.569610e+04,2.000000,0.000000,0.000000,-2016.000000,-0.160017,1.000000,0.000000,0.000000,48.000000,3720.000000,756.000000,-4284.000000,-0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007353,0.370931,0.000000
50%,1.428000e+04,1.604400e+04,12516.000000,21.000000,8.364100e+04,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,60.000000,8316.000000,3960.000000,2856.000000,0.311475,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.052632,0.663212,0.000000
75%,2.697600e+04,2.966400e+04,24720.000000,30.000000,2.165908e+05,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,84.000000,18144.000000,9960.000000,12264.000000,0.698784,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.273114,0.894737,1.000000
max,1.066024e+06,1.152000e+06,684400.000000,40.000000,7.469087e+06,5.000000,4.000000,4.000000,704250.000000,227.840000,1.000000,1.000000,1.000000,200.000000,177600.000000,92520.000000,533148.000000,84.160000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2027.000000,3227.000000,1.000000
median,1.428000e+04,1.604400e+04,12516.000000,21.000000,8.364100e+04,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,60.000000,8316.000000,3960.000000,2856.000000,0.311475,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.052632,0.663212,0.000000
skew,8.384717e+00,8.330773e+00,4.629575,-0.057879,5.391452e+00,-1.062876,2.099746,7.589147,-1.432409,-81.490345,-1.464529,1.660418,6.223143,0.316000,2.575573,3.037040,2.774865,-109.454552,-0.285668,4.359009,0.478129,2.485597,10.429182,2.639168,0.296179,0.664243,-0.178625,3.521235,4.235589,78.357635,109.920815,0.654727


In [11]:
def describe_num(df, stats):
    d = df.describe()
    return pd.concat([d, df.reindex(d.columns, axis = 1).agg(stats)])

In [12]:
num_cols = data.select_dtypes(exclude=['object', 'datetime64'])
num_cols_noncategorical = num_cols.loc[:, num_cols.nunique() >= 36]
describe_num(num_cols_noncategorical, ['median', 'skew', 'kurtosis'])

,SCHEDULED_PRODUCTION_QUANTITY,QUANTITY_START,RM_REQUIRED,WEEK,AVAILABLE_INVENTORY,Scheduled-Start,Scheduled-Start_%,Cases per Pallet,EPQ,SS,Scheduled-EPQ,Scheduled-EPQ_%,D_SCH_ST,SEPQ
count,3.386000e+04,3.386000e+04,33860.000000,33860.000000,3.386000e+04,33860.000000,33860.000000,33860.000000,31306.000000,31510.000000,31306.000000,31306.000000,33860.000000,31306.000000
mean,2.449780e+04,2.682506e+04,19454.171292,20.434613,1.546158e+05,-2327.267516,-0.760189,63.290992,15400.382163,7754.732402,5327.486742,-0.787896,0.807869,1.508157
std,4.326193e+04,4.556004e+04,25155.497595,10.807956,2.144682e+05,15126.826354,18.891933,29.145823,17360.304698,10787.572822,25732.290263,21.976232,18.273069,21.938573
min,-8.304000e+03,1.000000e+00,-4800.000000,1.000000,0.000000e+00,-611328.000000,-2207.000000,0.000000,22.000000,0.000000,-172600.000000,-3227.000000,-201.760000,-84.160000
25%,7.056000e+03,8.446500e+03,5040.000000,12.000000,1.569610e+04,-2016.000000,-0.160017,48.000000,3720.000000,756.000000,-4284.000000,-0.500000,0.007353,0.370931
50%,1.428000e+04,1.604400e+04,12516.000000,21.000000,8.364100e+04,0.000000,0.000000,60.000000,8316.000000,3960.000000,2856.000000,0.311475,0.052632,0.663212
75%,2.697600e+04,2.966400e+04,24720.000000,30.000000,2.165908e+05,0.000000,0.000000,84.000000,18144.000000,9960.000000,12264.000000,0.698784,0.273114,0.894737
max,1.066024e+06,1.152000e+06,684400.000000,40.000000,7.469087e+06,704250.000000,227.840000,200.000000,177600.000000,92520.000000,533148.000000,84.160000,2027.000000,3227.000000
median,1.428000e+04,1.604400e+04,12516.000000,21.000000,8.364100e+04,0.000000,0.000000,60.000000,8316.000000,3960.000000,2856.000000,0.311475,0.052632,0.663212
skew,8.384717e+00,8.330773e+00,4.629575,-0.057879,5.391452e+00,-1.432409,-81.490345,0.316000,2.575573,3.037040,2.774865,-109.454552,78.357635,109.920815


In [13]:
data_cat_stats = data.describe(include='object')
data_cat_stats

,JOB_STATUS,FG_CODE,LINE_CODE,RM_ITEM_CODE,JOB_CREATION_FLAG,RM Item Coverage,Job Coverage,Plant,LMS
count,33860,33860,33860,33860,33860,33860,33860,33860,33860
unique,3,1032,105,951,3,3,3,34,4
top,Closed,GRV05L40PDMN,RI3,SHV01CLR21525C,OUTSIDE LOCK WIDNOW,Covered,Covered,RIA,No
freq,33124,1694,1105,1842,31078,28218,26932,2828,18436


In [14]:
data_imputed = data.copy()

In [15]:
for col in data_imputed[['Low', 'Medium', 'High']]:
    proportion = data_imputed[col].value_counts(normalize=True)
    data_imputed[col] = data_imputed[col].fillna(pd.Series(np.random.choice([0.0, 1.0], p=proportion, size=len(data_imputed[col]))))

In [16]:
describe_num(data[['EPQ', 'SS', 'Scheduled-EPQ', 'Scheduled-EPQ_%', 'SEPQ']], ['median', 'skew', 'kurtosis'])

,EPQ,SS,Scheduled-EPQ,Scheduled-EPQ_%,SEPQ
count,31306.000000,31510.000000,31306.000000,31306.000000,31306.000000
mean,15400.382163,7754.732402,5327.486742,-0.787896,1.508157
std,17360.304698,10787.572822,25732.290263,21.976232,21.938573
min,22.000000,0.000000,-172600.000000,-3227.000000,-84.160000
25%,3720.000000,756.000000,-4284.000000,-0.500000,0.370931
50%,8316.000000,3960.000000,2856.000000,0.311475,0.663212
75%,18144.000000,9960.000000,12264.000000,0.698784,0.894737
max,177600.000000,92520.000000,533148.000000,84.160000,3227.000000
median,8316.000000,3960.000000,2856.000000,0.311475,0.663212
skew,2.575573,3.037040,2.774865,-109.454552,109.920815


In [17]:
# Only showing one example here. I tested everything more thor oughly in another notebook.
imputer = IterativeImputer(min_value=data_imputed[['EPQ', 'SS', 'Scheduled-EPQ', 'Scheduled-EPQ_%', 'SEPQ']].min(), tol=1e-5, sample_posterior=True)
data_imputed[['EPQ', 'SS', 'Scheduled-EPQ', 'Scheduled-EPQ_%', 'SEPQ']] = imputer.fit_transform(data_imputed[['EPQ', 'SS', 'Scheduled-EPQ', 'Scheduled-EPQ_%', 'SEPQ']])

In [18]:
describe_num(data_imputed[['EPQ', 'SS', 'Scheduled-EPQ', 'Scheduled-EPQ_%', 'SEPQ']], ['median', 'skew', 'kurtosis'])

,EPQ,SS,Scheduled-EPQ,Scheduled-EPQ_%,SEPQ
count,33860.000000,33860.000000,33860.000000,33860.000000,33860.000000
mean,15923.447957,8098.697358,5084.636514,-0.798035,1.506923
std,17201.189087,10694.791229,25668.658190,21.138720,21.103233
min,22.000000,0.000000,-172600.000000,-3227.000000,-84.160000
25%,4032.000000,1008.000000,-4872.000000,-0.631579,0.363834
50%,8640.000000,5304.715258,2784.000000,0.272534,0.674798
75%,21780.000000,11424.000000,12725.795052,0.696035,0.922064
max,177600.000000,92520.000000,533148.000000,84.160000,3227.000000
median,8640.000000,5304.715258,2784.000000,0.272534,0.674798
skew,2.439869,2.884656,2.583447,-113.707605,114.181623
